# Using the Retrieval Augmented Generation (RAG) Framework to contextualize the response of a chatbot.


## Imports


In [1]:
from dotenv import find_dotenv, load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_elasticsearch.vectorstores import ElasticsearchStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage
from langchain.chains import create_history_aware_retriever
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain_core.tools import tool
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.agents import  AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain import hub

In [5]:
load_dotenv(find_dotenv(), override=True)

True

## VectorDB setup and add embeddings to vector db


In [11]:
loader = PyPDFLoader("MG_Manual.pdf")#, extract_images=True)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                      chunk_overlap=512,
                                      length_function=len,
                                      is_separator_regex=False,
                                      )

docs = text_splitter.split_documents(documents)

In [12]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_db = ElasticsearchStore.from_documents(
    docs,
    es_url="http://localhost:9200",
    index_name="manual",
    embedding=embedding
)

vector_db.client.indices.refresh(index="manual")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

## Connecting vector db to LLM


In [14]:
retriever = vector_db.as_retriever()


In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
#llm = ChatOpenAI(name="gpt-4o",
#                 temperature=0)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [17]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
Keep your answers short and concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        #("system", qa_system_prompt),
        ("system", "{context}"),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [18]:
chat_history = []

i = 0
while i < 3:
    question = input("Ask a question")
    response = rag_chain.invoke({"input":question, "chat_history":chat_history})
    chat_history.extend([HumanMessage(content=question), response["answer"]])
    print(response["answer"])
    i += 1


This is a Table of Contents from a **vehicle manual**.  It covers topics like:

* **Tyre Maintenance:** How to check tread wear, rotate tyres, and information about winter tyres and chains.
* **Spare Tyre and Tools:**  Explains how to use the jack, where the lifting points are, and how to change a tyre.
* **Emergency Procedures:**  Includes instructions for jump-starting a dead battery, getting better traction in difficult conditions, and what to do if the vehicle gets trapped. 
* **Vehicle Care:**  Covers recommended cleaning agents and basic interior maintenance. 

Essentially, this manual seems to focus on the practical aspects of owning and maintaining the vehicle, especially in everyday situations and emergencies. 

You got it! It seems like a manual focused on the practical side of vehicle ownership, particularly emphasizing: 

* **Keeping the vehicle running:** Tyre care, changing a tyre, jump starting.
* **Handling common emergencies:**  Traction problems, getting stuck.
* **Ba

# Extending to Function Calling


## Setup Vector DB search retrieval as a tool.


In [19]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [20]:
@tool
def query_manual(query):
    """
    Queries the manual and retrieves information from its contents. 
    Returns the result and the source documents.

    Args:
        query (string): A query derived from the question asked by the user.
    """
    result = qa.invoke(query)
    return result['result'], result['source_documents']

In [21]:
from langchain import hub

template = """
You are an assistant that answers questions on the manual provided.
Use the tools provided to respond accurately. 
The query_manual tool should be used to retrieve information from the manual.

For questions that require further information, use the tavily_search_tool_json tool to conduct research and respond
with accurate answers.

Question: {input}
"""

prompt_template = PromptTemplate.from_template(template=template)

agentprompt = hub.pull("hwchase17/react-chat")

tools = [query_manual, TavilySearchResults(max_results=3)]

agent = create_react_agent(llm=llm,
                           tools=tools,
                           prompt=agentprompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               handle_parsing_errors=True,
                               verbose=True)

In [23]:
print(agentprompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'} template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assi

In [24]:
chat_history = []

i = 0
while i < 3:
    question = input("Ask a question")
    response = agent_executor.invoke({"input":prompt_template.format(input=question), "chat_history": chat_history})
    chat_history.extend({"user": question, "ai": response["output"]})
    print(response['output'])
    i += 1



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: query_manual
Action Input: What is this manual about?('The manual seems to be about car maintenance. \n', [Document(page_content='ŸMaintenance ............................................168\nŸInternal Tread Wear Indicator ...........169\nŸTyre Rotation ..........................................170\nŸWinter Tyres ..........................................170\nŸTyre Chain ...............................................170\n Spare Tyre, Jack and Tools ................171\nŸTool Kit ....................................................171\nŸSpare Tyre ...............................................171\nŸJack Lifting Point .....................................171\nŸTyre Replacement ..................................172\n Jump Start (Dead Battery) ................174\n Vehicle Traction ..................................176\nŸEmergency Trailer ..................................177 Getting Vehicle Out of Trap